# Assignment 9: CSV filter

Create Python program called `csvfilter.py` that will filter a delimited text file for some value, possibly in one of the columns.

In [ ]:
# Run this cell to make sure this assignment is up to date
%cd ~/be434-Spring2024
!git pull --no-edit upstream main

## Getting Started with new.py

Let's start out by using new.py to create a program template for us.


In [ ]:
# Generate the `csvfilter.py` using `new.py`
%cd ~/be434-Spring2024/assignments/09_csvfilter
!new.py -p 'Parse a CSV file' csvfilter.py

You should see the following:

```
$ new.py -p 'Parse a CSV file' csvfilter.py
Done, see new script "csvfilter.py."
```

## Instructions

### Usage

The program should take the following parameters:

* `-f`|`--file`: A *required* argument that is a readable file
* `-v`|`--val`: A *required* "value" to match against each record
* `-c`|`--col`: An optional "column" to search for the given value
* `-o`|`--outfile`: An optional output file name (default `'out.csv'`)
* `-d`|`--delimiter`: An optional delimiter to use to parse the file (default `','`)

Here is the expected usage for the program:

```
$ ./csvfilter.py -h
usage: csvfilter.py [-h] -f FILE -v val [-c col] [-o OUTFILE] [-d delim]

Filter delimited records

optional arguments:
  -h, --help            show this help message and exit
  -f FILE, --file FILE  Input file (default: None)
  -v val, --val val     Value for filter (default: None)
  -c col, --col col     Column for filter (default: )
  -o OUTFILE, --outfile OUTFILE
                        Output filename (default: out.csv)
  -d delim, --delimiter delim
                        Input delimiter (default: ,)
```

### Output

The program will search the `--file` for the `--val` value either in the given `--col` column or anywhere on the line in a _case-insensitive_ fashion.
Any records matching will be written to the `--outfile`.
The input files are delimited by commas and tabs, so you will need to use the `--delimiter` option to parse them.


## Time to write some code!

Open the script here in VS Code in be434-Spring2024 -> assignments -> 09_csvfilter -> csvfilter.py 

Write/edit the code using the instructions above.

## Hints: Parsing and writing delimited files

For this assignment, you will need to use the `csv` module, specifically:

* `csv.DictReader`
* `csv.DictWriter`

Be sure to read the https://docs.python.org/3/library/csv.html[documentation]!

Your `args.file` should be an open file handle, so you can create a reader:

```
reader = csv.DictReader(args.file, delimiter=args.delimiter)
```

This object has a `fieldnames` attribute which you should use to verify that the given `--col` argument is actually a valid option.
If it is not, your program should print an error message (preferably to `STDERR`) and exit _with an error value_.
You may optionally display the valid fieldnames, but this is not tested:

```
$ ./csvfilter.py -d $'\t' -v BACTERIA -f inputs/centroids.tab -c clas
--col "clas" not a valid column!
Choose from centroid, domain, kingdom, phylum, class, order, genus, species
```

You can use the `csv.DictWriter` to create a writer which can be used to write the header row to the output file:

```
writer = csv.DictWriter(args.outfile, fieldnames=reader.fieldnames)
writer.writeheader()
```

You can use a `for` loop to iterate through each record in the input file where the record will be represented as a `dict` having the first row column headers for the keys and the current record's data as the values.
Try this to start:

```
for rec in reader:
    print(rec)
	break
```

You will search for the indicated `--val` in all the `rec.values()` or just the given `--col` column from the record.
I would suggest you use a regular expression with the case-insensitive option:

```
if re.search(search_for, text, re.IGNORECASE):
    # write the output
```

You can refer to the `csv.DictWriter` documentation to see how to use the `writer` to write the record in a way that is similar to how we have used `SeqIO.write()` to write a sequence record to an output file handle.

When you are done, be sure to let the user know how many records were written to which output file name.

## The datasets

Let's start out by getting to know the datasets we will be working with, and learning how we can test our code along the way.

### Titanic

For the following, We can use commands from csvkit (a program installed on the HPC) to test the output of our program on various test files.
We will use the csvlook and csvgrep commands in csvkit.

In [ ]:
# let's look at the first three rows in the titanic file
%cd ~/be434-Spring2024/assignments/09_csvfilter
!csvlook --max-rows 3 inputs/titanic.csv

You should see something like the following:

----
| id | survived | pclass | sex    | age | sibsp | parch |    fare | embarked | class | who   | adult_male | deck | embark_town | alive | alone |
| -- | -------- | ------ | ------ | --- | ----- | ----- | ------- | -------- | ----- | ----- | ---------- | ---- | ----------- | ----- | ----- |
|  0 |    False |      3 | male   |  22 |     1 |     0 |  7.250… | S        | Third | man   |       True |      | Southampton | False | False |
|  1 |     True |      1 | female |  38 |     1 |     0 | 71.283… | C        | First | woman |      False | C    | Cherbourg   |  True | False |
|  2 |     True |      3 | female |  26 |     0 |     0 |  7.925… | S        | Third | woman |      False |      | Southampton |  True |  True |
----

In [ ]:
# If wanted to find how many records have the value "true" (case-insensitive), I could use `grep`:
!grep -i true inputs/titanic.csv | wc -l

You should see the following:

```
$ grep -i true inputs/titanic.csv | wc -l
     664
```

So if I run my program to search for this value, I should get the same number:

In [ ]:
# Try running your code to see if you get the following:
# Done, wrote 664 to "out.csv".
!./csvfilter.py -v true -f inputs/titanic.csv

This type of search is looking for the string "true" (case-insensitive) _anywhere on the line_.
If I wanted to only look in the `adult_male` column, however, I would get a different number:

In [ ]:
# Run the code with a column filter:
# Done, wrote 537 to "out.csv".
!./csvfilter.py -v true -c adult_male -f inputs/titanic.csv

In [ ]:
# We can confirm this number with the command csvgrep and word count
# note the extra count for the header, and that this is case-sensitive
!csvgrep -c adult_male -m "True" inputs/titanic.csv | wc -l

In [ ]:
# we can also use awk to validate this
# Note that `adult_male` is in column 12, so I can use that with `awk` to extract the 12th column and `grep` for "true"
# We should get 537, that looks legit!
!awk -F',' '{print $12}' inputs/titanic.csv | grep -i true | wc -l

## Centroids

Let's now check out the `centroids.txt` file:

# Look at centroids file
!csvlook --tabs --max-rows 3 inputs/centroids.tab

You should see something like this
| centroid                         | domain   | kingdom        | phylum              | class            | order                   | genus                | species              |
| -------------------------------- | -------- | -------------- | ------------------- | ---------------- | ----------------------- | -------------------- | -------------------- |
| e5d49c0803f04032b482a1ee836e18ab | Bacteria | Proteobacteria | Alphaproteobacteria | Rhodospirillales | AEGEAN-169 marine group | uncultured bacterium | uncultured bacterium |
| 2a4e004a710b7a1f3e676c0f4a5cfbb2 | Archaea  | Euryarchaeota  | Thermoplasmata      | Marine Group II  | uncultured archaeon     | uncultured archaeon  | uncultured archaeon  |
| d690a57bf58faabbac9effb929f66cb2 | Bacteria | Proteobacteria | Alphaproteobacteria | SAR11 clade      |                         |                      |                      |
| ...                              | ...      | ...            | ...                 | ...              | ...                     | ...                  | ...                  |



In [ ]:
# The string "bacteria" occurs on 493 lines:
!grep -i bacteria inputs/centroids.tab | wc -l

In [ ]:
# To parse this file, we'll need to indicate that the `--delimiter` is a tab character:
# Test your code and see if you get this:
# Done, wrote 493 to "bacteria.csv".
!./csvfilter.py -d $'\t' -v BACTERIA -f inputs/centroids.tab -o bacteria.csv

In [ ]:
# check this with word count
!wc -l bacteria.csv

In [ ]:
# If, however, we limit out search to the "class" column,
# the string "bacteria" occurs only 50 times:
# You should see this:
# Done, wrote 50 to "bacteria.csv".
!./csvfilter.py -d $'\t' -v BACTERIA -f inputs/centroids.tab -o bacteria.csv -c class

In [ ]:
# check this with word count
!wc -l bacteria.csv

## Testing

As you write your code, you can test it along the way to make sure that you are passing all of the tests for the homework. 
We will use the test suite that is included with the assignment to test that you are meeting all of the requirements in the instructions above. 
You will find the steps below to test your code. Note that you can also run these commands from a "shell" within the VS Code GUI. Or, you can run them here... 

In [ ]:
# Format your code to make it beautiful (this is called linting)
!black ~/be434-Spring2024/assignments/09_csvfilter/csvfilter.py

In [ ]:
# Now run the tests on your code
!make test

A passing test suite:

```
$ conda activate /groups/bhurwitz/bh_class/be434/be434-conda
$ make test
pytest -xv --disable-pytest-warnings test.py
=========================== test session starts ============================
...

test.py::test_exists PASSED                                           [ 12%]
test.py::test_usage PASSED                                            [ 25%]
test.py::test_bad_file PASSED                                         [ 37%]
test.py::test_bad_column PASSED                                       [ 50%]
test.py::test_titanic_any_true PASSED                                 [ 62%]
test.py::test_titanic_adult_male_true PASSED                          [ 75%]
test.py::test_centroids_any_bacteria PASSED                           [ 87%]
test.py::test_centroids_class_bacteria PASSED                         [100%]

============================ 8 passed in 0.51s =============================
```



## Uploading your code to GitHub

Once you have written the code for your assignment, and are passing all of the tests above, you are ready to submit the assignment for grading. Use the steps below to submit your code to GitHub.

* Note, if you are having any issues with passing tests, and need help, you can also submit the code with a different commit message like the following. 

```
git commit -m "test_XXX failing for 09_csvfilter"
```

Once you have done that, send a private slack message to me @bhurwitz to let me know you submitted code and need help.


In [ ]:
# Submit your code to Github
%cd
%cd be434-Spring2024
!git add -A && git commit -m "Submitting 09_csvfilter for grading"
!git push

Great job! You are done with this assignment.

## Authors

Bonnie Hurwitz <bhurwitz@arizona.edu> and Ken Youens-Clark <kyclark@gmail.com>